# Build Mutagenesis Libraries with SEAM
For each sequence in the activity library, generate a mutagenesis library using squid's RandomMutagenesis.

**Kernel**: Use SEAM_revisions venv (`/grid/wsbs/home_norepl/pmantill/SEAM_revisions/SEAM_revisions/.venv/`)

In [1]:
import os
import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import squid

2026-02-28 17:23:11.109459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-28 17:23:13.348220: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-28 17:23:14.353075: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/9.2.0/lib:/cm/lo

In [2]:
# Config
MODEL_NAME = "second_2stageig"  # which activity library to use
LIB_SIZE = 25000       # mutants per sequence
MUT_RATE = 0.10        # 10% mutation rate
SEQ_LENGTH = 230       # enhancer length (no promoter/barcode)
SEED = 42

ACTIVITY_LIB_DIR = Path("/grid/wsbs/home_norepl/pmantill/LentiMPRA_mcs/LentiMoCon/lenti_AGFT/motif_context_swap/library_prep/activity_libraries")
ACTIVITY_LIB_PATH = ACTIVITY_LIB_DIR / MODEL_NAME / f"k562_activity_library_{MODEL_NAME}.csv"

OUT_DIR = ACTIVITY_LIB_DIR / MODEL_NAME / "mutagenesis_libraries"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Model: {MODEL_NAME}")
print(f"Mutants per seq: {LIB_SIZE}, mutation rate: {MUT_RATE}")
print(f"Output: {OUT_DIR}")

Model: second_2stageig
Mutants per seq: 25000, mutation rate: 0.1
Output: /grid/wsbs/home_norepl/pmantill/LentiMPRA_mcs/LentiMoCon/lenti_AGFT/motif_context_swap/library_prep/activity_libraries/second_2stageig/mutagenesis_libraries


In [3]:
# Load activity library
lib_df = pd.read_csv(ACTIVITY_LIB_PATH)
print(f"Loaded {len(lib_df)} sequences from {ACTIVITY_LIB_PATH.name}")
print(f"Bins: {lib_df['activity_bin'].value_counts().to_dict()}")

# Verify all sequences are 230bp enhancers
seq_lens = lib_df["sequence"].str.len()
assert (seq_lens == SEQ_LENGTH).all(), f"Expected all seqs {SEQ_LENGTH}bp, got: {seq_lens.describe()}"
print(f"All sequences are {SEQ_LENGTH}bp (enhancer only, no promoter/barcode)")

Loaded 3000 sequences from k562_activity_library_second_2stageig.csv
Bins: {'high': 1000, 'mid': 1000, 'low': 1000}
All sequences are 230bp (enhancer only, no promoter/barcode)


In [4]:
# One-hot encoding (ACGT order)
ALPHA_MAP = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def str_to_onehot(seq_str):
    ohe = np.zeros((len(seq_str), 4), dtype=np.float32)
    for j, base in enumerate(seq_str):
        if base in ALPHA_MAP:
            ohe[j, ALPHA_MAP[base]] = 1.0
    return ohe

# Initialize mutagenizer
mut_generator = squid.mutagenizer.RandomMutagenesis(mut_rate=MUT_RATE, seed=SEED)
print(f"Mutagenizer ready: ~{int(np.ceil(MUT_RATE * SEQ_LENGTH))} mutations per sequence")

Mutagenizer ready: ~23 mutations per sequence


In [5]:
# Generate mutagenesis libraries for each sequence
for i, row in lib_df.iterrows():
    seq_id = row["seq_id"]
    activity_bin = row["activity_bin"]
    out_file = OUT_DIR / f"{activity_bin}_{seq_id}.h5"

    if out_file.exists():
        if (i + 1) % 100 == 0:
            print(f"  [{i+1}/{len(lib_df)}] {seq_id} — already exists, skipping")
        continue

    wt_onehot = str_to_onehot(row["sequence"])
    x_mut = mut_generator(wt_onehot, num_sim=LIB_SIZE)

    with h5py.File(out_file, 'w') as f:
        f.create_dataset('sequences', data=x_mut, dtype='float32',
                         compression='gzip', compression_opts=4)
        f.create_dataset('wt_sequence', data=wt_onehot, dtype='float32')
        f.attrs['seq_id'] = seq_id
        f.attrs['activity_bin'] = activity_bin
        f.attrs['actual_activity'] = row['actual_activity']
        f.attrs['split'] = row['split']
        f.attrs['fold'] = int(row['fold'])
        f.attrs['n_mutants'] = LIB_SIZE
        f.attrs['mut_rate'] = MUT_RATE
        f.attrs['seq_length'] = SEQ_LENGTH
        f.attrs['alphabet'] = 'ACGT'

    if (i + 1) % 100 == 0 or i == 0:
        print(f"  [{i+1}/{len(lib_df)}] {activity_bin}/{seq_id} — {x_mut.shape} saved")

print(f"\nDone! Generated mutagenesis libraries in {OUT_DIR}")

Mutagenesis: 100%|██████████| 25000/25000 [00:01<00:00, 23183.74it/s]


  [1/3000] high/ENSG00000111885_Reversed: — (25000, 230, 4) saved


Mutagenesis: 100%|██████████| 25000/25000 [00:01<00:00, 24098.94it/s]


  [100/3000] high/peak54698 — (25000, 230, 4) saved


Mutagenesis: 100%|██████████| 25000/25000 [00:00<00:00, 27482.51it/s]


...

  [2800/3000] low/ENSG00000170634 — (25000, 230, 4) saved


Mutagenesis: 100%|██████████| 25000/25000 [00:00<00:00, 30777.00it/s]


  [2900/3000] low/peak85019_Reversed: — (25000, 230, 4) saved


Mutagenesis: 100%|██████████| 25000/25000 [00:00<00:00, 25724.65it/s]


  [3000/3000] low/ENSG00000166971 — (25000, 230, 4) saved

Done! Generated mutagenesis libraries in /grid/wsbs/home_norepl/pmantill/LentiMPRA_mcs/LentiMoCon/lenti_AGFT/motif_context_swap/library_prep/activity_libraries/second_2stageig/mutagenesis_libraries


In [6]:
# Summary: verify outputs
h5_files = sorted(OUT_DIR.glob("*.h5"))
print(f"Total H5 files: {len(h5_files)}")
print(f"Expected: {len(lib_df)}")

# Spot check first file
if h5_files:
    with h5py.File(h5_files[0], 'r') as f:
        print(f"\nSpot check: {h5_files[0].name}")
        print(f"  sequences shape: {f['sequences'].shape}")
        print(f"  wt_sequence shape: {f['wt_sequence'].shape}")
        print(f"  attrs: {dict(f.attrs)}")

# Count by bin
bin_counts = {}
for f_path in h5_files:
    bin_name = f_path.stem.split('_')[0]
    bin_counts[bin_name] = bin_counts.get(bin_name, 0) + 1
print(f"\nFiles per bin: {bin_counts}")

Total H5 files: 3000
Expected: 3000

Spot check: high_ENSG00000000460.h5
  sequences shape: (25000, 230, 4)
  wt_sequence shape: (230, 4)
  attrs: {'activity_bin': 'high', 'actual_activity': 1.331, 'alphabet': 'ACGT', 'fold': 10, 'mut_rate': 0.1, 'n_mutants': 25000, 'seq_id': 'ENSG00000000460', 'seq_length': 230, 'split': 'test'}

Files per bin: {'high': 1000, 'low': 1000, 'mid': 1000}
